In [2]:
import sys
import os
from time import time

DATAPATH = '/Users/balthasar/Google Drive/MTL_data/wsj/'
sys.path.append(DATAPATH)
sys.path.append('../')

from src.util import count_lines_many, data_from_many, unpack, bunch_paragraphs

datafiles = [DATAPATH + file for file in os.listdir(DATAPATH)]
print("Number of entries: %i" % (count_lines_many(datafiles)))

Number of entries: 46726


In [71]:
import re 
from bs4 import BeautifulSoup

data = data_from_many(datafiles)
url, html = unpack(next(data))

In [72]:
def get_paragraphs(html):
    soup = BeautifulSoup(html,'html.parser')
    paragraphs = [tag.get_text() for tag in 
        soup.findAll('title')+soup.findAll("h1",'wsj-article-headline')+soup.findAll("h2",'sub-head')+soup.findAll("p")]
    if len(paragraphs) != 0:
        return paragraphs
    else:
        return 'NOCONTENT'
    
label_re = re.compile('(?<=<meta name="article.type" content=").*(?=" />)')
def get_label(html):
    label = label_re.search(html)
    if label:
        return '/'.join(label.group().split(' - '))
    else:
        return "NOLABEL"
     

def preprocess(item):
    url, html = unpack(item)
    label = get_label(html)
    paragraphs = get_paragraphs(html)
    return label, paragraphs

In [73]:
# Class count -- count only entries that have easily identifiable content and label
from collections import Counter
from multiprocessing import Pool

classes = Counter()
subclasses = Counter()

i,j = (0,0)
word_count = 0
paragraph_count = 0

def analyze_item_one(item):
    global i,j,word_count,paragraph_count,classes,subclasses
    i+=1
    label,paragraphs = item
    if label != 'NOLABEL' and paragraphs != 'NOCONTENT':
        j+=1
        class_ = label.split('/')[0]
        subclass_ = label
        
        paragraph_count += len(paragraphs)
        for paragraph in paragraphs:
            word_count += len(paragraph.split(' '))

        classes.update([class_])
        subclasses.update([subclass_])
        
        print("[%i/%i] Word Count: %i Paragraph Count: %i Top-10 classes: %s" % (j,i,word_count,paragraph_count,str(classes.most_common(10))),end='\r')

def analyze_item_many(data):
    list(map(analyze_item_one,data))
    
analyze_item_many(data) 

ValueError: too many values to unpack (expected 2)

In [63]:
data = map(preprocess,data_from_many(datafiles))
analyze_item_many(data)        
with open('../summaries/wsj.txt','w') as file:
    file.write('WSJ\n')
    file.write(50*"#")
    file.write('\nWORD COUNT: %i\n' % word_count)
    file.write('PARAGRAPH COUNT: %i\n' % paragraph_count)
    file.write('CLASSES: \n')
    for item in list(classes.items()):
        file.write('%s\t%i\n' % item)
    file.write('\n\n')
    file.write('SUBCLASSES: \n')
    for item in list(subclasses.items()):
        file.write('%s\t%i\n' % item)
    file.write('\n\n')

In [66]:
# get a datasample for Friederike
import json

data = map(preprocess,data_from_many(datafiles))
i = 0
with open('wsj_datasample.csv','w') as file:
    for label, paragraphs in data:
        if label != 'NOLABEL' and paragraphs != 'NOCONTENT':
            text_chunks = bunch_paragraphs(paragraphs,target_length=250)
            labels = [label for i in range(len(text_chunks))]
            items = zip(labels,text_chunks)
            for item in items:
                if len(item[1].split(' ')) >= 100:
                    print(i,end='\r')
                    i+=1
                    file.write(json.dumps(item)+'\n')
                    if i == 3000:
                        raise RateLimit

NameError: name 'RateLimit' is not defined

In [76]:
preprocess(next(data))

('NBA',
 ['\n    James Harden’s Stepback 3-Pointer Is a Step Ahead of the NBA - WSJ\n  ',
  'James Harden’s Stepback 3-Pointer Is a Step Ahead of the NBA',
  'The Houston Rockets star has a revolutionary shot of his own. He’s taking more stepback 3-pointers by himself than any NBA team. And it’s working.',
  'NATHANIEL BUTLER/NBAE/GETTY IMAGES',
  'By',
  '\nBen Cohen\n',
  ' Houston ',
  'The most spectacular play of this NBA season began like so many other Houston Rockets possessions. James Harden isolated his defender on the perimeter. The Rockets spread themselves across the court to let him do the one thing he does better than anyone in the history of basketball.',
  'Before he could take yet another stepback 3-pointer,...\n',
  ' Houston ',
  'The most spectacular play of this NBA season began like so many other Houston Rockets possessions. James Harden isolated his defender on the perimeter. The Rockets spread themselves across the court to let him do the one thing he does bette